# Data Crawling

In [1]:
from bs4 import BeautifulSoup
import urllib.request

In [2]:
#- Data Crawling

raw_file = open('./data/raw_review_data.txt', 'w', encoding='UTF-8') # Create file for writing

for no in range(1, 501):
    url = 'https://movie.naver.com/movie/point/af/list.naver?&page=%d' % no
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    reviews = soup.select('tbody > tr > td.title')
    for rev in reviews: 
        rev_lst = []
        title = rev.select_one('a.movie').text.strip()
        score = rev.select_one('div.list_netizen_score > em').text.strip()
        comment = rev.select_one('br').next_sibling.strip()
        
        #- labeling
        if int(score) >= 8 :
            label = 1   # 8 ~ 10 score -> Positive Reviews
        elif int(score) <= 4 :
            label = 0   # 0 ~ 4 score -> Negative Reviews
        else :
            label = 2   # 5 ~ 7 score -> Normal Reviews
        
        raw_file.write(f'{title}\t{score}\t{comment}\t{label}\n')
raw_file.close()

In [3]:
#- Data Structure
import pandas as pd

In [4]:
raw_txt_data = pd.read_csv('./data/raw_review_data.txt', delimiter = '\t', names=['title', 'score', 'comment', 'label'])
raw_data = pd.DataFrame(raw_txt_data)
raw_data.head(10)

,title,score,comment,label
0,3000년의 기다림,10,엔딩크레딧과 함께 흐르는 음악까지 완벽,1
1,영웅,1,그냥 전개가 아예없고 그냥 뮤지컬을 화면으로 옮겼다 라는 느낌? 내용과 하등 관련없...,0
2,솔캠,10,NaN,1
3,세 자매,8,마지막 배우들의 미친 연기력에 소름 돋다 눈물이 또르르...안봐야지 하다 봤는데 나...,1
4,더 퍼스트 슬램덩크,10,팬들을 위한 존중과 배려가 돋보이는 영화이노우에 다케히코가 리얼이후 현실적인 묘사들...,1
5,아바타: 물의 길,10,3D 이녀석 아주 성장했구만~! 열분 IMAX3D로 감상하면 돈 값하는 영화의 맛을...,1
6,아바타: 물의 길,8,체험하는 영화 조금 졸았음,1
7,영웅,4,지루함... 너무나 훌륭한 주제지만 그렇다고 창작물 까지도 좋다고 할 수는 없..,0
8,내 머리 속의 지우개,10,여기가 천국인가요 ㅠㅠ,1
9,온리 더 브레이브,10,NaN,1


In [6]:
#- Data Purification
raw_data = raw_data.dropna() # Removal NaN values
raw_data = raw_data.drop_duplicates(['comment']) # Removal overlaping values

In [8]:
#- Check Null
raw_data.isnull().sum()

title      0
score      0
comment    0
label      0
dtype: int64

In [12]:
#- Removal unnecessary Column
raw_data = raw_data.drop(['title', 'score'],axis=1)

In [13]:
#- Save
raw_data.to_csv("./data/review_sample_data.csv", encoding="utf-8")